# Recurrent Neural Networks

## Part 1-Introduction to RNNs

### What are RNNS
    make use of sequential information
    Here is what a typical RNN looks like:
    
    

![title](img/rnn1.png)

![unfold](img/unfold.jpg)

### Example: Character-level language model sampling
   Vocabulary: **[h,e,l,o]**
   
   Examplel training sequence: **"hello"**
   
   ![charseq](img/charseq.jpeg)

At test-time **sample** characters one at a time, feed back to model
![test-time](img/test-time.png)

Backpropagation through time
![BPTT](img/BBTT.png)
Impossible to train!

**Truncated** Backpropagation through time
![TBPTT](img/TBPTT.png)

## Part 2-Implementation with python
[min-char-rnn.py](https://gist.github.com/karpathy/d4dee566867f8291f086) gist:112 lines of python

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

data has 4214 characters, 60 unique.


In [2]:
smooth_loss

102.35861405555251

In [3]:
while smooth_loss>50:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 n"Wkf6 .OBcbCpLMc
LOMhtx)Tuiy(Mgt2
4mj-nc)yktywSywgj:,gcbo
LkojWSBo0r0rVL!lIzIjRpw,-Fjdrth6hnWx,0 s.!V
qTa-wFiOxIslg9
M5e23BAps,7hc4AicAotxLROva
sI3x21kOA,b)Mwxw4MurSM'vFS.yg2acm'wfSx)157qWd pynRa.TMo 
----
iter 0, loss: 102.358608
----
 ,hlvd  hig icaoid  ilrh t hghe ioaiardhnhspotmioooi  enh oiotpptef tniesrwucetctnbrpMeluur'   shrpovumvsniylcneveg sashn ir  rcuns iipe'hn elthenejva
ci reitti p eeW tnydi,rndheieewhcw ovh ediyuo aokl 
----
iter 100, loss: 102.022735
----
 arctiitl  yahiutsu eprnsihlb,skrtonsmmvuarwar oroas ed  se senors,  etoeebte l hwrwredee ,yudhuneilritypseysniliose stdi vevidhuaeIfueao iS ggia nroV,e gotbrddoat vtefsepdsdmndnt pe wndco b aapnmhagtd 
----
iter 200, loss: 100.052819
----
 s o
3lieesolhs m3wl a oa sehintianeinte eitml estee mn d   i tlsod nlasapetseefroeet mseut tl ae orforsta  ataoe9v ol enRro tntra nnon per.eltoa ad  tenpyte iigp tlae i n  eaiwun p rehoAeen mlsadbrl b 
----
iter 300, loss: 98.043792
----
 . eiWalbotwn sAmspts tsee stsu9rmwoat

## Part 3-Long Short Term Memory (LSTM)
- Exploding gradients
    - Truncated BPTT
    - Clip gradients at threshold (something like anti-windup in control science LOL)
    - RMSProp to adjust learning rate
- Vanishing gradients
    - Harder to detect
    - Weight Initialization
    - ReLU activation functions
    - RMSProp
    - LSTM, GRUs (<– That’s why we use LSTM)
    
###  Introduction
   LSTM is proposed in [Hochreiter et al., 1997]. GRU is a knid of simplified LSTM.
   ![LSTM_diagram](img/LSTM_diagram.png)
   

LSTM have two states, one is cell state (c), another is hidden state (h):
  - *i*: input gate, “add to memory”, decides whether do we want to add value to this cell.
  - *f*: forget gate, “flush the memory”, decides whether to shut off the cell and reset the counter.
  - *o*: output gate, “get from memory”, decides how much do we want to get from this cell.
  - *g*: input, decides how much do we want to add to this cell.
  ![LSTM_structure](img/LSTM_structure.png)

## Part 4-LSTM implementation

## Part 5-Resources
[Recurrent neural networks(Video)](https://www.youtube.com/watch?v=Ukgii7Yd_cU)

[Notes for CS231n Recurrent Neural Network(Post)](http://www.yuthon.com/2016/10/30/Notes-for-CS231n-RNN/)

[Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

[Simple LSTM](http://nicodjimenez.github.io/2014/08/08/lstm.html)